In [1]:
import requests
from bs4 import BeautifulSoup
import xmltodict
import random
import fasttext
from collections import Counter
import time
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import re
import razdel
import matplotlib.pyplot as plt
import numpy as np
from razdel import sentenize, tokenize
from tqdm.auto import tqdm, trange

C:\Users\Admin\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def id2paragraphs(idx, lang='bxr'):
    soup = BeautifulSoup(requests.get(f'https://{lang}.wikipedia.org/?curid={idx}').text)
    body = soup.find('div', {'id': 'bodyContent'})
    return [paragraph.text for paragraph in body.findAll('p')]

In [3]:
base_url = 'https://dumps.wikimedia.org/bxrwiki/'
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, 'html.parser')
# Find the links on the page
dumps = [a['href'] for a in soup_index.find_all('a') if 
         a.has_attr('href')]
dumps

['../',
 '20221120/',
 '20221201/',
 '20221220/',
 '20230101/',
 '20230120/',
 '20230201/',
 '20230220/',
 '20230301/',
 '20230320/',
 '20230401/',
 'latest/']

In [4]:
dump_url = base_url + '20230320/'
# Retrieve the html
dump_html = requests.get(dump_url).text
# Convert to a soup
soup_dump = BeautifulSoup(dump_html, 'html.parser')
# Find list elements with the class file
soup_dump.find_all('li', {'class': 'file'})[:3]

[<li class="file"><a href="/bxrwiki/20230320/bxrwiki-20230320-pages-articles-multistream.xml.bz2">bxrwiki-20230320-pages-articles-multistream.xml.bz2</a> 5.1 MB</li>,
 <li class="file"><a href="/bxrwiki/20230320/bxrwiki-20230320-pages-articles-multistream-index.txt.bz2">bxrwiki-20230320-pages-articles-multistream-index.txt.bz2</a> 65 KB</li>,
 <li class="file"><a href="/bxrwiki/20230320/bxrwiki-20230320-pages-logging.xml.gz">bxrwiki-20230320-pages-logging.xml.gz</a> 980 KB</li>]

In [5]:
files = []

# Search through all files
for file in soup_dump.find_all('li', {'class': 'file'}):
    text = file.text
    # Select the relevant files
    if 'pages-articles' in text:
        files.append((text.split()[0], text.split()[1:]))
        
files[:5]

[('bxrwiki-20230320-pages-articles-multistream.xml.bz2', ['5.1', 'MB']),
 ('bxrwiki-20230320-pages-articles-multistream-index.txt.bz2', ['65', 'KB']),
 ('bxrwiki-20230320-pages-articles.xml.bz2', ['4.8', 'MB'])]

In [6]:
import xml.sax

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))

In [7]:
# Content handler for Wiki XML
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

handler._pages

[]

In [8]:
import bz2
import subprocess

data_path = 'bxrwiki-20230320-pages-articles.xml.bz2'

In [9]:
lines = []
for i, line in enumerate(bz2.BZ2File(data_path, 'r')):
    lines.append(line)
    if i > 1e6:
        break

In [10]:
for l in lines[:20]:
    parser.feed(l)

In [11]:
# !pip install wiki_dump_parser
# import pandas as pd
# df = pd.read_csv('bxrwiki-20230320-pages-articles.csv', quotechar='|', index_col = False)
# df['timestamp'] = pd.to_datetime(df['timestamp'],format='%Y-%m-%dT%H:%M:%SZ')
# df

In [12]:
with open('bxrwiki-20230320-pages-meta-current.xml', 'r', encoding = 'utf-8') as f:
    raw = f.read()

In [13]:
struct = xmltodict.parse(raw)

In [14]:
struct['mediawiki'].keys()

dict_keys(['@xmlns', '@xmlns:xsi', '@xsi:schemaLocation', '@version', '@xml:lang', 'siteinfo', 'page'])

In [15]:
len(struct['mediawiki']['page'])

11077

In [16]:
page = struct['mediawiki']['page'][0]
page = random.choice(struct['mediawiki']['page'])
page
page['revision']['text']['#text']

'#перенаправление [[8 һарын 7]]'

In [17]:
with open('bxrwiki-20230320-langlinks.sql', 'r', encoding = 'utf-8') as f:
    rl = f.readlines()

In [18]:
def get_content(soup):
    body = soup.find('div', {'id': 'bodyContent'})
    return [paragraph.text for paragraph in body.findAll('p')]

def parse_by_id(idx, lang='bxr'):
    retry_strategy = Retry(
      total=3,
      backoff_factor=1
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    http = requests.Session()
    http.mount("https://", adapter)
    http.mount("http://", adapter)
    url = f'https://{lang}.wikipedia.org/?curid={idx}'
    soup = BeautifulSoup(http.get(url).text)
    results = {
        'url': url,
        'content': get_content(soup)
    }
    ru_button = soup.find('li', {'class': 'interlanguage-link interwiki-ru mw-list-item'})
    if ru_button:
        ru_url = ru_button.find('a')['href']
        ru_soup = BeautifulSoup(requests.get(ru_url).text)
        results['ru_url'] = ru_url
        results['ru_content'] = get_content(ru_soup)
    return results

In [19]:
page = random.choice(struct['mediawiki']['page'])
print(f'https://bxr.wikipedia.org/?curid={page["id"]}')
out = parse_by_id(page['id'])
print(out['content'][:3])
if out.get('ru_url'):
    print(out['ru_url'])
    print(out['ru_content'][:3])
else:
    print('no ru')

https://bxr.wikipedia.org/?curid=1598
['Hello\xa0:) My name is Kelovy, I live in Bratislava, capital of Slovakia. \n']
https://ru.wikipedia.org/wiki/%D0%A3%D1%87%D0%B0%D1%81%D1%82%D0%BD%D0%B8%D0%BA:Kelovy
['privet iz Bratislavy\n']


In [21]:
# all_results = []
# i = 0
for j in trange(5, 11):
    all_results = []
    i = 1
    for page in tqdm(struct['mediawiki']['page'][j*1000:(j+1)*1000]):
        if i % 100 == 0:
            time.sleep(random.randint(5, 10))
        try:
            all_results.append(parse_by_id(page['id']))
        except AttributeError:
            pass
        i += 1
    with open(f'wiki_parsed_{j}.json', 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False)

100%|████████████████████████████████████████████████████████████████████████████████| 6/6 [1:46:55<00:00, 1069.26s/it]


In [26]:
all_results = []
i = 1
for page in tqdm(struct['mediawiki']['page'][4908:4999]):
    if i % 100 == 0:
        time.sleep(random.randint(5, 10))
    try:
        all_results.append(parse_by_id(page['id']))
    except AttributeError:
        pass
    i += 1
    with open(f'wiki_parsed_2.json', 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 91/91 [01:10<00:00,  1.30it/s]


In [28]:
len(struct['mediawiki']['page'])

11077

In [29]:
all_results = []
i = 1
for page in tqdm(struct['mediawiki']['page'][11000:11077]):
    if i % 100 == 0:
        time.sleep(random.randint(5, 10))
    try:
        all_results.append(parse_by_id(page['id']))
    except AttributeError:
        pass
    i += 1
    with open(f'wiki_parsed_3.json', 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [00:59<00:00,  1.29it/s]


In [24]:
wiki_full = []
with open('wiki_1.json', 'r', encoding='utf-8') as f:
        wiki_full += json.load(f)
for i in range(5,11):
    with open(f'wiki_parsed_{i}.json', 'r', encoding='utf-8') as f:
        wiki_full += json.load(f)

In [25]:
len(wiki_full)

10908

In [ ]:
for i in range(2,4):
    with open(f'wiki_parsed_{i}.json', 'r', encoding='utf-8') as f:
        wiki_full += json.load(f)

In [31]:
len(wiki_full)

11076

In [32]:
# with open('wiki_full.json', 'w', encoding='utf-8') as f:
#     json.dump(wiki_full, f, ensure_ascii=False)

# Extract pages only with buryat content

In [ ]:
# with open('wiki_parsed_1.json', 'w', encoding='utf-8') as f:
#     json.dump(all_results, f, ensure_ascii=False)

In [2]:
class LanguageDetector:
    def __init__(self, path="lid.323.ftz"):
        self.model = fasttext.load_model(path)

    def predict_lang(self, text, k=10):
        text = text.replace('\n', '  ')
        langs, proba = self.model.predict(text, k=k)
        res = Counter(dict(zip([lang[9:] for lang in langs], proba)))
        for key in ['ru', 'bxr']:
            if key not in res:
                res[key] = 0
        return res
    
LD = LanguageDetector()

In [3]:
with open('wiki_full.json', 'r', encoding='utf-8') as f:
    all_results = json.load(f)

In [4]:
print('all: ', len(all_results))
print('rus: ', len([r for r in all_results if 'ru_content' in r]))

all:  11076
rus:  6686


In [5]:
good_results = {}
for item in tqdm(all_results):
    url = item['url']
    text = '\n\n'.join(item['content']).strip()
    text = re.sub('\[\d+\]', '', text)

    pars = []
    bxr_scores = []
    ru_scores = []
    top_langs = []
    lens = []
    for p in text.split('\n\n'):
        p = p.strip()
        if len(p.strip()) < 3:
            continue
        if (p.count('•') + p.count('·') + p.count('|')) / len(p) > 0.05:
            continue
        if not re.match('.*[а-яёһүө].*', p.lower(), re.DOTALL):
            continue
        pars.append(p)
        langs = LD.predict_lang(p)
        bxr_scores.append(langs['bxr'])
        ru_scores.append(langs['ru'])
        top_langs.append(langs.most_common(1)[0][0])
        lens.append(len(p))
    
    good_pars = '\n\n'.join([p for i, p in enumerate(pars) if top_langs[i] == 'bxr'])
    
    if good_pars:
        good_results[url] = good_pars
        
print(len(good_results))

100%|███████████████████████████████████████████████████████████████████████████| 11076/11076 [00:19<00:00, 571.56it/s]

5235


In [38]:
url = random.choice(list(good_results.keys()))
print(url)
text = good_results[url]
print(text)

https://bxr.wikipedia.org/?curid=10416
Хиин гагнуури (автоген гагнуури) юрын гагнуурида адли галай хүсээр улайлган шэрээжэ гагнадаг бэшэ, хабшамал хүшэлтүрэгшэ, гагнаха гэһэн газар тиишэ хандуулан табижа, түүнэй хүсээр гагнадагые иижэ нэрлэнэ.


In [39]:
# with open('clean_bxr.json', 'w', encoding='utf-8') as f:
#     json.dump(good_results, f, ensure_ascii=False)

# extract semi-aligned texts

In [7]:
url2item = {item['url']: item for item in all_results}

In [8]:
candidates = list({k for k, v in good_results.items() if 'ru_content' in url2item[k]})
len(candidates)

4522

In [10]:
with open('bur_ru_wiki_all.json', 'w', encoding='utf-8') as f:
    json.dump(candidates, f, ensure_ascii=False)

In [11]:
def get_good_text(paragraphs, target_language='ru'):
    text = '\n\n'.join(paragraphs).strip()
    text = re.sub('\[\d+\]', '', text)

    pars = []
    top_langs = []
    for p in text.split('\n\n'):
        p = p.strip().replace('\xa0', ' ')
        if len(p.strip()) < 3:
            return
        if (p.count('•') + p.count('·') + p.count('|')) / len(p) > 0.05:
            return
        if not re.match('.*[а-яё].*', p.lower(), re.DOTALL):
            return
        pars.append(p)
        langs = LD.predict_lang(p)
        top_langs.append(langs.most_common(1)[0][0])
    
    good_pars = '\n\n'.join([p for i, p in enumerate(pars) if top_langs[i] == target_language])
    
    return good_pars

In [12]:
import torch
from transformers import AutoTokenizer, AutoModel

In [13]:
mname = 'labse_bur_tokenizer'
tokenizer = AutoTokenizer.from_pretrained(mname)
model = AutoModel.from_pretrained(mname)

Some weights of the model checkpoint at labse_bur_tokenizer were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
def embed(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='pt')
    with torch.inference_mode():
        model_output = model(**encoded_input.to(model.device))
    embeddings = model_output.pooler_output
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [15]:
def center_norm(v):
    v = v - v.mean(0)
    return v /  (v**2).sum(1, keepdims=True) ** 0.5


def center_dot(x, y):
    m = (x.sum(0) + y.sum(0)) / (x.shape[0] + y.shape[0])
    x = x - m
    y = y - m
    x =  x /  (x**2).sum(1, keepdims=True) ** 0.5
    y =  y /  (y**2).sum(1, keepdims=True) ** 0.5
    return np.dot(x, y.T)

In [16]:
def get_top_mean_by_row(x, k=5):
    m, n = x.shape
    k = min(k, n)
    topk_indices = np.argpartition(x, -k, axis=1)[:, -k:]
    rows, _ = np.indices((m, k))
    return x[rows, topk_indices].mean(1)

In [17]:
def align3(sims):
    
    #sims = np.dot(center_norm(orig_vecs), center_norm(sum_vecs).T) ** 3
    #sims = center_dot(orig_embeds, sum_embeds) #** 3

    rewards = np.zeros_like(sims)
    choices = np.zeros_like(sims).astype(int)  # 1: choose this pair, 2: decrease i, 3: decrease j

    # алгоритм, разрешающий пропускать сколько угодно пар, лишь бы была монотонность
    for i in range(sims.shape[0]):
        for j in range(0, sims.shape[1]):
            # вариант первый: выровнять i-тое предложение с j-тым
            score_add = sims[i, j]
            if i > 0 and j > 0:  # вот как тогда выровняются предыдущие 
                score_add += rewards[i-1, j-1]
                choices[i, j] = 1
            best = score_add
            if i > 0 and rewards[i-1, j] > best:
                best = rewards[i-1, j]
                choices[i, j] = 2
            if j > 0 and rewards[i, j-1] > best:
                best = rewards[i, j-1]
                choices[i, j] = 3
            rewards[i, j] = best
    alignment = []
    i = sims.shape[0] - 1
    j = sims.shape[1] - 1
    while i > 0 and j > 0:
        if choices[i, j] == 1:
            alignment.append([i, j])
            i -= 1
            j -= 1
        elif choices[i, j] == 2:
            i -= 1
        else:
            j -= 1
    return alignment[::-1]

In [149]:
url = random.choice(candidates)
sents_bur = [s.text for p in good_results[url].split('\n') for s in razdel.sentenize(p)  if s.text]
sents_ru = [s.text for p in get_good_text(url2item[url]['ru_content']).split('\n') for s in razdel.sentenize(p)  if s.text]
print(sents_bur)
print(sents_ru)
emb_ru = np.stack([embed(s) for s in tqdm(sents_ru)])
emb_er = np.stack([embed(s) for s in tqdm(sents_bur)])

['Зургадугаар һарын 30 — Григориин литын жэлэй 181-дэхи үдэр (үндэр жэлдэ 182-дохи үдэр).', 'Жэлэй эсэс болотор 184 үдэрнүүд үлэжэ байна.']
['30 июня — 181-й день года (182-й в високосные годы) по григорианскому календарю.', 'До конца года остаётся 184 дня.', 'До 15 октября 1582 года — 30 июня по юлианскому календарю, с 15 октября 1582 года — 30 июня по григорианскому календарю.', 'В XX и XXI веках соответствует 17 июня по юлианскому календарю.', 'Один из двух дней года, в которые может быть добавлена високосная секунда (другой такой день — 31 декабря)[источник не указан 146 дней].', 'См. также: Категория:События 30 июня', 'См. также: Категория:Родившиеся 30 июня', 'См. также: Категория:Умершие 30 июня']



100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.20it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.60it/s]


In [150]:
pen = np.array([[min(len(x), len(y)) / max(len(x), len(y)) for x in sents_bur] for y in sents_ru])
sims = np.maximum(0, np.dot(emb_ru, emb_er.T)) ** 1 * pen

alpha = 0.2
penalty = 0.2
sims_rel = (sims.T - get_top_mean_by_row(sims) * alpha).T - get_top_mean_by_row(sims.T) * alpha - penalty

alignment = align3(sims_rel)

print('total score: ', round(sum(sims[i, j] for i, j in alignment) / min(sims.shape), 2))
print('alignment:\n')
for i, j in alignment:
    print(sents_ru[i])
    print(sents_bur[j])
    print(round(sims[i, j], 2))
    print('-')

total score:  0.19
alignment:

До конца года остаётся 184 дня.
Жэлэй эсэс болотор 184 үдэрнүүд үлэжэ байна.
0.38
-


total score 0.19
sim 0.38 - хорошо

sim 0.37

До 15 октября 1582 года — 18 июня по юлианскому календарю, с 15 октября 1582 года — 18 июня по григорианскому календарю.
XV зуун жэлэй Томас де Торквемадагай эмхидхэһэн Испаниин инквизици Филиппын ударидалга доро эгээн хүсэтэй болобо.
0.39811444431543347
-
В XX и XXI веках соответствует 5 июня по юлианскому календарю.
1876 ондо Изабелла хатанай хүбүүн XII Альфонсо хаан болоһон.
0.3691283445204458
-


In [30]:
sents_ru

['Ширинга — посёлок в Еравнинском районе Бурятии.',
 'Административный центр сельского поселения «Ширингинское».',
 'Расположен на северо-восточном берегу Малого Еравного озера в 22 км к северо-востоку от районного центра, села Сосново-Озёрское, по западной стороне межрегиональной автодороги Р436 Улан-Удэ — Романовка — Чита.',
 'Климат резко континентальный, характеризуется малоснежной зимой с сильными морозами, а летом жаркими днями и прохладными ночами.',
 'Среднегодовая температура отрицательна, равна -1,9 С°.',
 'Наиболее холодными месяцами являются январь и февраль, жаркими — июнь и июль.',
 'Абсолютная минимальная температура равна -55 С°, максимальная — +32 С°.',
 'Национальный состав населения: буряты – 343 чел., русские – 170 чел.',
 'Также в посёлке проживают татары, казахи, узбеки, цыгане, китайцы, украинцы.',
 'Средняя общеобразовательная школа, детский сад, Дом культуры, фельдшерско-акушерский пункт, почтовое отделение.',
 'Земли в районе посёлка представлены мерзлотными л

In [ ]:
wiki_pairs = []
tq = tqdm(candidates)
for url in tq:
    ru_text = get_good_text(url2item[url]['ru_content'])
    if not ru_text: 
        continue
    sents_er = [s.text for p in good_results[url].split('\n') for s in razdel.sentenize(p)  if s.text]
    sents_ru = [s.text for p in ru_text.split('\n') for s in razdel.sentenize(p)  if s.text]
    
    emb_ru = np.stack([embed(s) for s in sents_ru])
    emb_er = np.stack([embed(s) for s in sents_er])
    
    pen = np.array([[min(len(x), len(y)) / max(len(x), len(y)) for x in sents_er] for y in sents_ru])
    sims = np.maximum(0, np.dot(emb_ru, emb_er.T)) ** 1 * pen

    alpha = 0.2
    penalty = 0.2
    sims_rel = (sims.T - get_top_mean_by_row(sims) * alpha).T - get_top_mean_by_row(sims.T) * alpha - penalty

    alignment = align3(sims_rel)

    total_score = sum(sims[i, j] for i, j in alignment) / min(sims.shape)
    if total_score < 0.15:
        continue
    
    for i, j in alignment:
        if sims[i, j] >= 0.50: # порог высоковат; часть предложений мы потеряем, но полученные зато будут чистыми
            wiki_pairs.append([sents_er[j], sents_ru[i]])
    tq.set_description(str(len(wiki_pairs)))

387:  39%|███████████████████████████▏                                         | 1783/4522 [4:35:03<3:48:14,  5.00s/it]

In [ ]:
for p in wiki_pairs:
    p[0] = p[0].replace('\xa0', ' ')

In [ ]:
random.choice(wiki_pairs)

In [ ]:
with open('wiki_aligned.json', 'w') as f:
    json.dump(wiki_pairs, f, ensure_ascii=False)